In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA

Loading data

In [3]:
train_bike = pd.read_csv("train.csv")
#test_bike = pd.read_csv("test.csv")

In [4]:
train_bike["datetime"].head()

0    2011-01-01 00:00:00
1    2011-01-01 01:00:00
2    2011-01-01 02:00:00
3    2011-01-01 03:00:00
4    2011-01-01 04:00:00
Name: datetime, dtype: object

In [5]:
train_bike["datetime"] = pd.to_datetime(train_bike["datetime"])
train_bike["datetime"].describe()

count                   10886
unique                  10886
top       2011-06-09 04:00:00
freq                        1
first     2011-01-01 00:00:00
last      2012-12-19 23:00:00
Name: datetime, dtype: object

In [6]:
train_bike["dayofweek"] = train_bike["datetime"].dt.dayofweek      #The day of the week with Monday=0, Sunday=6
train_bike["year"] = train_bike["datetime"].dt.year
train_bike["month"] = train_bike["datetime"].dt.month
train_bike["day"] = train_bike["datetime"].dt.day
train_bike["hour"] = train_bike["datetime"].dt.hour


In [7]:
X = train_bike[["season", "holiday", "workingday", "weather","temp","humidity","windspeed"]]
y = train_bike["registered"]

In [8]:
from sklearn.model_selection import train_test_split #dzielnie na zetsawy
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, random_state=42)

In [9]:
X_train

,season,holiday,workingday,weather,temp,humidity,windspeed
1101,1,0,1,3,17.22,0,15.0013
4194,4,0,1,1,21.32,59,7.0015
10389,4,0,0,1,11.48,81,12.9980
6969,2,0,0,1,25.42,22,19.9995
313,1,0,1,1,9.02,37,26.0027
6037,1,0,1,1,15.58,43,15.0013
3557,3,0,1,1,27.06,69,19.0012
4614,4,0,0,1,11.48,56,22.0028
5665,1,0,1,1,8.20,80,8.9981
118,1,0,1,2,6.56,64,6.0032


In [10]:
from sklearn.svm import LinearSVR

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
best_score = 0
parameters = {}

In [13]:
from tqdm import tqdm

In [14]:
for C in tqdm([1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.]):
    for tol in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
        for dual in [True, False]:
            for epsilon in [1e-4, 1e-3, 1e-2, 1e-1, 1]:
                for loss in ["epsilon_insensitive", "squared_epsilon_insensitive"]:
                    clf = LinearSVR(C = C, tol = tol, dual = dual, loss = loss, epsilon = epsilon)
                    try:
                        scores = cross_val_score(clf, X_train, y_train, cv = 5)
                    except ValueError as e:
                        #print(e)
                        continue
                    if scores.mean() > best_score:
                        parameters["C"] = C
                        parameters["tol"] = tol
                        parameters["dual"] = dual
                        parameters["loss"] = loss
                        parameters["epsilon"] = epsilon
                        best_score = scores.mean()
                        print("Accuracy: {}".format(scores.mean()))
                        print("C: {}, tol: {}, dual: {}, loss: {}, epsilon: {}".format(C, tol, dual, loss,epsilon))

  0%|          | 0/11 [00:00<?, ?it/s]

Accuracy: 0.15880898333297955
C: 0.0001, tol: 1e-05, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.15882055080197288
C: 0.0001, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.15882143562530487
C: 0.0001, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.15884601103083412
C: 0.0001, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.15885094706438824
C: 0.0001, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.15885291989717426
C: 0.0001, tol: 0.01, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.01
Accuracy: 0.15895810386507392
C: 0.0001, tol: 0.1, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.0001


  9%|▉         | 1/11 [00:03<00:35,  3.57s/it]

Accuracy: 0.1825943312781847
C: 0.001, tol: 1e-05, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.1825944314680782
C: 0.001, tol: 1e-05, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.18260147677914979
C: 0.001, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.1826107011351114
C: 0.001, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.01
Accuracy: 0.18261773273186582
C: 0.001, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.1826294756739834
C: 0.001, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.1827205753543548
C: 0.001, tol: 0.001, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.1
Accuracy: 0.18277250714512422
C: 0.001, tol: 0.01, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.001


 18%|█▊        | 2/11 [00:12<00:54,  6.05s/it]

Accuracy: 0.18979015917133935
C: 0.01, tol: 1e-05, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.18981302634562278
C: 0.01, tol: 1e-05, dual: True, loss: squared_epsilon_insensitive, epsilon: 1
Accuracy: 0.18981353869156997
C: 0.01, tol: 0.01, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.18986836031852794
C: 0.01, tol: 0.01, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.1
Accuracy: 0.1901031575282104
C: 0.01, tol: 0.1, dual: True, loss: squared_epsilon_insensitive, epsilon: 0.001


 27%|██▋       | 3/11 [00:56<02:31, 18.91s/it]

Accuracy: 0.1902397049456796
C: 0.1, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.1902426481034365
C: 0.1, tol: 1e-05, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.001
Accuracy: 0.19025512952034526
C: 0.1, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.0001
Accuracy: 0.1902721809110781
C: 0.1, tol: 0.0001, dual: False, loss: squared_epsilon_insensitive, epsilon: 0.1


100%|██████████| 11/11 [11:22<00:00, 62.07s/it]


In [19]:
parameters

{'C': 0.1,
 'tol': 0.0001,
 'dual': False,
 'loss': 'squared_epsilon_insensitive',
 'epsilon': 0.1}

In [20]:
clf = LinearSVR(C = parameters["C"], tol = parameters["tol"], dual = parameters["dual"], epsilon = parameters["epsilon"], 
                loss = parameters["loss"])

In [21]:
clf.fit(X_train, y_train)

LinearSVR(C=0.1, dual=False, epsilon=0.1, fit_intercept=True,
     intercept_scaling=1.0, loss='squared_epsilon_insensitive',
     max_iter=1000, random_state=None, tol=0.0001, verbose=0)

In [24]:
y_pred = clf.predict(X_test)

In [25]:
y_pred

array([242.98455914,  34.93077408, 139.7302894 , ..., 249.42034234,
       281.74025079, 244.69542052])

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

AttributeError: 'LinearSVR' object has no attribute 'predict_proba'